<a href="https://colab.research.google.com/github/akiyoss-git/CollabMersedesNN/blob/main/NN_Mersedes_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Серебрянский Александр Сергеевич 18-АС (18-ИВТ-1)
#3 Вариант

##Подготовка, Импорты

In [ ]:
!pip install -U keras-tuner

In [ ]:
import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from kerastuner.engine import hyperparameters
from kerastuner import HyperModel
from keras.models import Sequential
import matplotlib.pyplot as plt
import IPython

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Подготовка данных

###Получение и вывод тренировочных данных

In [ ]:
train_path = '/content/drive/My Drive/labs/train.csv'
data = pd.read_csv(train_path)
data.head()

###Удаление столбика ID

In [ ]:
del data['ID']
data.head()

###Получение истинных значений и удаление стобца Y

In [ ]:
y_data = data.get('y')
x_data = data.drop('y' , axis=1)

###Кодирование буквенных данных

In [ ]:
x_data = (x_data - x_data.mean()) / x_data.std()
col = x_data.select_dtypes(include=[object])
x_data.loc[:, col.columns] = col.apply(LabelEncoder().fit_transform)
data.head()

###Разделение данных на тренировочные и тестовые

In [ ]:
split = 0.7
seed = 200

In [ ]:
x_train = x_data.sample(frac=split, random_state=seed)
x_train = x_train.drop('y', axis=1)
x_test = x_data.drop(x_train.index)
x_train = x_train.fillna(x_train.mean().mean())

y_train = y_data.sample(frac=split, random_state=seed)
y_test = y_data.drop(y_train.index)

In [ ]:
x_train

In [ ]:
y_train

##Создание и обучение нейросети

###Создание нейросети

In [ ]:
model = Sequential()
model.add(Dense(400, activation="relu", input_shape=(x_train.shape[1],)))
model.add(Dense(200, activation="relu"))
model.add(Dense(1))
print("[DEBUG-USER] nn created")

model.compile(optimizer="adam", loss="mse", metrics=["mae"])
print("[DEBUG-USER] nn compiled")

model.summary()

###Обучение нейросети

In [ ]:
history = model.fit(x_train, y_train, epochs=200, batch_size=1000, verbose=1, validation_split=0.3)
history = history.history
print("[DEBUG-USER] nn finish")

###Вывод графика

In [ ]:
def graphs(history):
    loss = history["loss"]
    val_loss = history["val_loss"]
    epochs = range(1, len(history['loss']) + 1)
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    plt.clf()

    mae = history['mae']
    val_mae = history['val_mae']
    plt.plot(epochs, mae, 'r', label='Training mae')
    plt.plot(epochs, val_mae, 'b', label='Validation mae')
    plt.title('Training and validation mae')
    plt.xlabel('Epochs')
    plt.ylabel('mae')
    plt.legend()
    plt.show()



# рисуем все графики
graphs(history)

###Предсказания нейросети

In [ ]:
x_test = x_test.drop('y', axis=1)

In [ ]:
predictions = model.predict(x_test, verbose=1)
predictions